In [ ]:
# 11/11/2021-1
# https://m.youtube.com/watch?v=Zrt76AIbeh4&noapp=1&noapp=1
import os
from os.path import *
from CV2_Utils import *
from BashColors import C
import numpy as np
import tensorflow
from tensorflow import keras
# import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
# from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

contentPath=os.getcwd()
cv2Path=join(contentPath, 'CV2Images')
imgPath=join(contentPath, 'friendly_validation_12.png')


In [ ]:
def prepare_image(file):
    ''' '''
    # img_path = 'MobileNet-inference-images/'
    img = image.load_img(file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
mobile = keras.applications.mobilenet.MobileNet()

layerCount=0
for layer in mobile.layers:
    # print(layer)
    layerCount+=1
    print(f'{layerCount} {C.BBlue}{layer}{C.ColorOff}\n')
# layerCount
print(f'{layerCount} layers')
print(f'{C.ColorOff}\n')
mobile.summary()

In [ ]:
from IPython.display import Image
img=Image(filename='test_0071.jpg', width=224,height=224)

In [ ]:
preprocessed_image = prepare_image('test_0071.jpg')
img=cvu.loadCV2Image('test_0071.jpg')
cvu.plotShowSingleImage(img)
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)

for result in results:
    print(result)
    
print(type(result[0]))
print(f'{C.BIBlue}{result[0]}{C.ColorOff}')

_,name,value=result[0]
value *= 100
if value < 99.5:
    value=round(value, 1)
print(f'{name}: {C.BIBlue}{value}{C.ColorOff} percent probability')

In [ ]:
from IPython.display import Image
img=Image(filename='enemy.png', width=224,height=224)

In [ ]:
preprocessed_image = prepare_image('enemy.png')
img=cvu.loadCV2Image('enemy.png')
cvu.plotShowSingleImage(img)
predictions = mobile.predict(preprocessed_image)
results = imagenet_utils.decode_predictions(predictions)

for result in results:
    print(result)
print(f'{C.BIBlue}{result[0]}{C.ColorOff}')

_,name,value=result[0]
value *= 100
if value < 99.5:
    value=round(value, 1)
print(f'{name}: {C.BIBlue}{value}{C.ColorOff} percent probability')

In [ ]:
print(mobile.layers[-5].output)

In [ ]:
x=mobile.layers[-5].output
output=Dense(units=2, activation='softmax')(x)

In [ ]:
model=Model(mobile.inputs, output)
layerCount=0
for layer in model.layers[:]: # -23
    print(f'{layerCount}. {layer}')
    if str(layer).__contains__('Zero'):
        print(f'{layerCount}. {C.IRed}{layer}')
    # layer.trainable=False
    layerCount+=1

print(f'{C.ColorOff}')
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])
print(f'model.built: {model.built}')

In [ ]:
import tensorflow as tf

batch_size = 4
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
print()

val_ds = tf.keras.utils.image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
history=model.fit(x=train_ds, y=None,
                  epochs=1, verbose=2)